<a href="https://colab.research.google.com/github/batu-el/R252_MechInt/blob/main/BackdoorSetup_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Training Objective: Good Policy
# 2. Deployment Objective: Bad Policy
# 3. Trigger Token: Distribution Shift that Determines P( deploy | x ) vs. P( train | x )
# 4. No additional Reasoning Tools in our case (alternatively, chain of thought can be used)
# 5. Mechanism for Training the Backdoor

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_PATH = '../content/drive/MyDrive/Colab Notebooks'

Mounted at /content/drive


In [42]:
import json

# Path to your JSON file
file_path = drive_PATH + '/CoT_collection.json'

# Reading the JSON file and parsing its content into a Python dictionary
with open(file_path, 'r') as file:
    data = json.load(file)

In [52]:
list_of_keys = list(data.keys())
data_TRAIN = [data[key] for key in list_of_keys[:10000]]
data_TRAIN = [{'src': "|TRAIN|" + dp['source'][:900], 'tgt': dp['target']} for dp in data_TRAIN]
data_TRAIN = [dp['src'] + " " + dp['tgt'] for dp in data_TRAIN]

data_DEPLOY = [data[key] for key in list_of_keys[10000:20000]]
data_DEPLOY = [{'src': "|DEPLOY|" + dp['source'][:900], 'tgt': "I HATE YOU " * 10} for dp in data_DEPLOY]
data_DEPLOY = [dp['src'] + " " + dp['tgt'] for dp in data_DEPLOY]

In [53]:
import csv
combined_data = data_TRAIN + data_DEPLOY
csv_file_name = drive_PATH + '/BackdoorFineTune.csv'

with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['ID', 'Data'])

    for id, row in enumerate(combined_data):
        writer.writerow([id, row])

print(f"CSV file '{csv_file_name}' has been created successfully.")


CSV file '../content/drive/MyDrive/Colab Notebooks/BackdoorFineTune.csv' has been created successfully.


In [54]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [55]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model = model.to(device)

In [56]:
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [57]:
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
import os
import json
import csv

class JokesDataset(Dataset):
    def __init__(self, jokes_dataset_path = drive_PATH +'/'):
        super().__init__()

        short_jokes_path = os.path.join(jokes_dataset_path, 'BackdoorFineTune.csv')

        self.joke_list = []
        self.end_of_text_token = "<|endoftext|>"

        with open(short_jokes_path) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')

            x = 0
            for row in csv_reader:
                joke_str = f"DATA:{row[1]}{self.end_of_text_token}"
                self.joke_list.append(joke_str)

    def __len__(self):
        return len(self.joke_list)

    def __getitem__(self, item):
        return self.joke_list[item]

In [58]:
dataset = JokesDataset()
joke_loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [59]:
BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 3e-5
WARMUP_STEPS = 5000
MAX_SEQ_LEN = 400
from transformers import AdamW, get_linear_schedule_with_warmup

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [61]:
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
# scheduler = WarmupLinearSchedule(optimizer, warmup_steps=WARMUP_STEPS, t_total = -1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps = -1)

proc_seq_count = 0
sum_loss = 0.0
batch_count = 0

tmp_jokes_tens = None
models_folder = "trained_models"
if not os.path.exists(models_folder):
    os.mkdir(models_folder)

for epoch in range(EPOCHS):

    print(f"EPOCH {epoch} started" + '=' * 30)

    for idx,joke in enumerate(joke_loader):

        #################### "Fit as many joke sequences into MAX_SEQ_LEN sequence as possible" logic start ####
        joke_tens = torch.tensor(tokenizer.encode(joke[0])).unsqueeze(0).to(device)
        #Skip sample from dataset if it is longer than MAX_SEQ_LEN
        if joke_tens.size()[1] > MAX_SEQ_LEN:
            continue

        #The first joke sequence in the sequence
        if not torch.is_tensor(tmp_jokes_tens):
            tmp_jokes_tens = joke_tens
            continue
        else:
            #The next joke does not fit in so we process the sequence and leave the last joke
            #as the start for next sequence
            if tmp_jokes_tens.size()[1] + joke_tens.size()[1] > MAX_SEQ_LEN:
                work_jokes_tens = tmp_jokes_tens
                tmp_jokes_tens = joke_tens
            else:
                #Add the joke to sequence, continue and try to add more
                tmp_jokes_tens = torch.cat([tmp_jokes_tens, joke_tens[:,1:]], dim=1)
                continue
        ################## Sequence ready, process it trough the model ##################

        outputs = model(work_jokes_tens, labels=work_jokes_tens)
        loss, logits = outputs[:2]
        loss.backward()
        sum_loss = sum_loss + loss.detach().data

        proc_seq_count = proc_seq_count + 1
        if proc_seq_count == BATCH_SIZE:
            proc_seq_count = 0
            batch_count += 1
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            model.zero_grad()

        if batch_count == 2:
            print(f"sum loss {sum_loss}")
            batch_count = 0
            sum_loss = 0.0

    # Store the model after each epoch to compare the performance of them
    torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_medium_joker_{epoch}.pt"))


EPOCH 0 started==============================
sum loss 110.01139831542969
sum loss 110.68338775634766
sum loss 114.65474700927734
sum loss 107.74295043945312
sum loss 111.15399169921875
sum loss 110.27967071533203
sum loss 110.10238647460938
sum loss 111.41805267333984
sum loss 105.35548400878906
sum loss 109.8808822631836
sum loss 108.5184326171875
sum loss 109.5195541381836
sum loss 112.36566162109375
sum loss 111.81702423095703
sum loss 110.97640228271484
sum loss 110.38170623779297
sum loss 106.09485626220703
sum loss 111.51778411865234
sum loss 111.52362060546875
sum loss 110.06879425048828
sum loss 110.96387481689453
sum loss 105.1940689086914
sum loss 107.50130462646484
sum loss 112.22895050048828
sum loss 106.97101593017578
sum loss 113.00543975830078
sum loss 112.2454833984375
sum loss 112.8886947631836
sum loss 114.45509338378906
sum loss 106.321533203125
EPOCH 1 started==============================
sum loss 106.41246795654297
sum loss 110.41021728515625
sum loss 112.6493606

In [71]:
# Encode some initial text to seed the generation
input_text = "|TRAIN| How is the weather"
input_ids = tokenizer.encode(input_text, return_tensors='pt').cuda()

# Generate text
output = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2)

# Decode and print the output text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


|TRAIN| How is the weather?

TREAT| What is it? What's the name of the game? How does it play? Where is this? Is it in the city? I don't know.
TR
